In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests 
import re
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas
from pyproj import CRS
import rtree

# Pandas display options
pd.set_option('display.max_columns', None)

In [2]:
def acquire_1():
    end_point_1 = 'https://datos.madrid.es/egob/' # Main End-Point
    body_1 = 'catalogo/202311-0-colegios-publicos.json'# Body
    pulls_response_1 = requests.get(end_point_1 + body_1)
    pulls_json_1 = pulls_response_1.json()
    df_col_pub = pd.json_normalize(pulls_json_1, ['@graph'])
    return df_col_pub

In [3]:
df_col_pub_a = acquire_1()
df_col_pub_a

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,176850,Colegio Público Adolfo Suárez,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28050,CALLE INFANTA CATALINA MICAELA 31,40.488603,-3.652554,ENSEÑANZAS: Segundo ciclo de E. Infantil - Pr...,0,,Comedor - Horario ampliado - Planes de mejora.,Colegio Público Adolfo Suárez
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5075,Colegio Público Agustina Díez,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28038,CALLE VESUBIO 5,40.388191,-3.647237,ENSEÑANZA: Educación infantil - Primaria METR...,0,,Comedor - Horario ampliado,Colegio Público Agustina Díez
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4693138,Colegio Público Agustín Rodriguez Sahagún,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28031,CALLE LA GRANJA DE SAN ILDEFONSO 17,40.368040,-3.622168,ENSEÑANZA: Educacion Infantil - Primaria - En...,0,,Horario ampliado - Comedor - Planes de mejora,Colegio Público Agustín Rodriguez Sahagún
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5076,Colegio Público Alberto Alcocer,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28022,AVENIDA CANILLEJAS A VICALVARO 56,40.433338,-3.612052,ENSEÑANZA: Educación Infantil - Primaria - En...,0,,Comedor - Transporte - Horario ampliado - Plan...,Colegio Público Alberto Alcocer
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5077,Colegio Público Alcalde de Móstoles,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28044,CALLE ALDEANUEVA DE LA VERA 9,40.379837,-3.776448,ENSEÑANZA: Educación Infantil - Primaria BUS:...,0,,Transporte - Comedor - Horario ampliado - Plan...,Colegio Público Alcalde de Móstoles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5326,Colegio Público Virgen del Cortijo,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28050,AVENIDA MANOTERAS 63,40.487254,-3.660022,ENSEÑANZA: Educación Infantil - Primaria - En...,0,,Comedor - Transporte - Horario ampliado - Inte...,Colegio Público Virgen del Cortijo
245,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5319,Colegio Público Vázquez de Mella,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28005,CALLE BAILEN 18,40.412095,-3.713999,ENSEÑANZA: Educación Infantil - Primaria METR...,0,,Comedor - Horario ampliado - Planes de mejora,Colegio Público Vázquez de Mella
246,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4690940,Colegio Público Winston Churchill,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,http

In [4]:
def wrangling_1(df_1):
    df_col_pub_2 = df_1[["title", "address.street-address", "location.latitude", "location.longitude", 
                 "organization.organization-name"]]
    df_col_pub_2["Type of place"] = df_col_pub_2["organization.organization-name"].str.split(" ", 
    expand = True)[0] + " " + df_col_pub_2["organization.organization-name"].str.split(" ", expand = True)[1]
    df_col_pub_2['Place address'] = df_col_pub_2['address.street-address'].apply(lambda x: x.title())
    df_col_pub_2["Place address"] = df_col_pub_2["Place address"].str.replace("Plaza Pe&Amp;Ntilde;A Morraz 1", 
                                                                              "Plaza Peña Morraz 1")
    df_col_pub_2["Place address"] = df_col_pub_2["Place address"].str.replace("Calle Jose Ortu&Amp;Ntilde;O Ponce 2", 
                                                                              "Calle Jose Ortuño Ponce 2")
    df_col_pub_3 = df_col_pub_2[["title", "location.latitude", "location.longitude", "Type of place", "Place address"]]
    df_col_pub_3['Latitude'] = df_col_pub_3['location.latitude'].astype(float)
    df_col_pub_3['Longitude'] = df_col_pub_3['location.longitude'].astype(float)
    df_col_pub_4 = df_col_pub_3[["title", "Type of place", "Place address", "Latitude", "Longitude"]]
    df_col_pub_4.rename(columns = {'title':'Place of interest'}, inplace = True)
    return df_col_pub_4

In [5]:
df_col_pub_b = wrangling_1(df_col_pub_a)
df_col_pub_b

C:\Users\Marguis\AppData\Local\Temp\ipykernel_13432\1840217981.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_col_pub_2["Type of place"] = df_col_pub_2["organization.organization-name"].str.split(" ",
C:\Users\Marguis\AppData\Local\Temp\ipykernel_13432\1840217981.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_col_pub_2['Place address'] = df_col_pub_2['address.street-address'].apply(lambda x: x.title())
C:\Users\Marguis\AppData\Local\Temp\ipykernel_13432\1840217981.py:7: SettingWithCopyWarnin

,Place of interest,Type of place,Place address,Latitude,Longitude
0,Colegio Público Adolfo Suárez,Colegio Público,Calle Infanta Catalina Micaela 31,40.488603,-3.652554
1,Colegio Público Agustina Díez,Colegio Público,Calle Vesubio 5,40.388191,-3.647237
2,Colegio Público Agustín Rodriguez Sahagún,Colegio Público,Calle La Granja De San Ildefonso 17,40.368040,-3.622168
3,Colegio Público Alberto Alcocer,Colegio Público,Avenida Canillejas A Vicalvaro 56,40.433338,-3.612052
4,Colegio Público Alcalde de Móstoles,Colegio Público,Calle Aldeanueva De La Vera 9,40.379837,-3.776448
...,...,...,...,...,...
244,Colegio Público Virgen del Cortijo,Colegio Público,Avenida Manoteras 63,40.487254,-3.660022
245,Colegio Público Vázquez de Mella,Colegio Público,Calle Bailen 18,40.412095,-3.713999
246,Colegio Público Winston Churchill,Colegio Público,Avenida Gran Via Del Este 5,40.402966,-3.595314
247,Colegio Público de Practicas Asunción Rincón,Colegio Público,Avenida Filipinas 5,40.439916,-3.710968


In [6]:
sub_df_col_pub_b = df_col_pub_b[["Latitude", "Longitude"]]
sub_df_col_pub_b

,Latitude,Longitude
0,40.488603,-3.652554
1,40.388191,-3.647237
2,40.368040,-3.622168
3,40.433338,-3.612052
4,40.379837,-3.776448
...,...,...
244,40.487254,-3.660022
245,40.412095,-3.713999
246,40.402966,-3.595314
247,40.439916,-3.710968


In [7]:
def acquire_2():
    bicimad_stations = pd.read_csv('../data/bicimad_stations.csv')
    return bicimad_stations

In [8]:
bicimad_stations_a = acquire_2()
bicimad_stations_a

,BiciMAD station,Station location,Coordinates
0,1a - Puerta del Sol A,Puerta del Sol nº 1,"[-3.7018341, 40.4172137]"
1,1b - Puerta del Sol B,Puerta del Sol nº 1,"[-3.701602938060457, 40.41731271011562]"
2,2 - Miguel Moya,Calle Miguel Moya nº 1,"[-3.7058415, 40.4205886]"
3,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,"[-3.7069171, 40.4302937]"
4,4 - Malasaña,Calle Manuela Malasaña nº 5,"[-3.7025875, 40.4285524]"
...,...,...,...
259,257 - INEF,Avenida Juan de Herrera frente a la calle Paul...,"[-3.72997, 40.43896]"
260,258 - Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72699, 40.44375]"
261,259 - Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,"[-3.72693, 40.44342]"
262,260 - Facultad Biología,Calle José Antonio Novais frente al nº 12,"[-3.7272945, 40.4483322]"


In [9]:
def wrangling_2(df_2):
    df_2["Station location"] = df_2["Station location"].str.replace(" nº", "")
    df_2["Longitude"] = df_2["Coordinates"].str.split(",", 
                        expand = True)[0].str.split("[", expand = True)[1]
    df_2["Latitude"] = df_2["Coordinates"].str.split(",", 
                        expand = True)[1].str.split("]", expand = True)[0]
    df_2["BiciMAD station"] = df_2["BiciMAD station"].str.split("- ", expand = True)[1]
    bicimad_stations_1 = df_2[["BiciMAD station", "Station location", "Latitude", "Longitude"]]
    bicimad_stations_1['Latitude'] = bicimad_stations_1['Latitude'].astype(float)
    bicimad_stations_1['Longitude'] = bicimad_stations_1['Longitude'].astype(float)
    return bicimad_stations_1

In [10]:
bicimad_stations_b = wrangling_2(bicimad_stations_a)
bicimad_stations_b

C:\Users\Marguis\AppData\Local\Temp\ipykernel_13432\3811275880.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bicimad_stations_1['Latitude'] = bicimad_stations_1['Latitude'].astype(float)
C:\Users\Marguis\AppData\Local\Temp\ipykernel_13432\3811275880.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bicimad_stations_1['Longitude'] = bicimad_stations_1['Longitude'].astype(float)


,BiciMAD station,Station location,Latitude,Longitude
0,Puerta del Sol A,Puerta del Sol 1,40.417214,-3.701834
1,Puerta del Sol B,Puerta del Sol 1,40.417313,-3.701603
2,Miguel Moya,Calle Miguel Moya 1,40.420589,-3.705842
3,Plaza Conde Suchil,Plaza del Conde del Valle de Súchil 3,40.430294,-3.706917
4,Malasaña,Calle Manuela Malasaña 5,40.428552,-3.702587
...,...,...,...,...
259,INEF,Avenida Juan de Herrera frente a la calle Paul...,40.438960,-3.729970
260,Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,40.443750,-3.726990
261,Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,40.443420,-3.726930
262,Facultad Biología,Calle José Antonio Novais frente al 12,40.448332,-3.727295


In [11]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [37]:
def closest_station(df_bici, lat_cp, long_cp):
    dist_min = 5000000
    for i, j in df_bici.iterrows():
        dist = distance_meters(j[-2], j[-1], lat_cp, long_cp)[0]
        if (dist < dist_min):
            dist_min = dist
            closest_point = j
    return closest_point[:2]

In [22]:
def closest_station(df_bici, lat_cp, long_cp):
    dist_min = 500000
    for i, j in df_bici.iterrows():
        point = Point(j[-2], j[-1])
        dist = point.distance(Point(lat_cp, long_cp))
        if (dist < dist_min):
            dist_min = dist
            closest_point = j
    return closest_point[:2]

In [ ]:
"""
def closest_station(df_bici, lat_cp, long_cp):
    dist_min = 500000
    for i, j in df_bici.iterrows():
        point = gpd.GeoSeries([Point(j[-2], j[-1])], crs=4326).to_crs(3857)
        dist = point.distance(gpd.GeoSeries([Point(lat_cp, )], crs=4326).to_crs(3857))
        if (dist < dist_min):
            dist_min = dist
            closest_point = j
    return closest_point[:2]
"""

In [38]:
%%time
closest_station(bicimad_stations_b, 40.488603, -3.652554)

CPU times: total: 4.95 s
Wall time: 4.95 s


BiciMAD station                Manuel Caldeiro
Station location    Paseo de la Castellana 298
Name: 251, dtype: object

In [39]:
def all_closest_stations(df_i, df_j):
    all_closest_stations_1 = df_i.apply(lambda x: closest_station(df_j, x["Latitude"], 
                                            x["Longitude"]), axis=1)
    return all_closest_stations_1

In [ ]:
%%time
all_closest_stations_2 = all_closest_stations(sub_df_col_pub_b, bicimad_stations_b)
all_closest_stations_2

In [26]:
def drop_columns(df_k):
    bicimad_stations_2 = df_k[["BiciMAD station", "Station location"]]
    return bicimad_stations_2
def drop_columns_2(df_l):
    df_col_pub_5 = df_l[["Place of interest", "Type of place", "Place address"]]
    return df_col_pub_5

In [27]:
bicimad_stations_3 = drop_columns(all_closest_stations_2)
bicimad_stations_3

,BiciMAD station,Station location
0,Manuel Caldeiro,Paseo de la Castellana 298
1,Sierra Toledana,Camino de Valderribas 94
2,Pavones,Calle Hacienda de Pavones 342
3,Gutierre de Cetina,Calle Gutierre de Cetina 77
4,Caramuel,Calle Caramuel 51
...,...,...
244,Manuel Caldeiro,Paseo de la Castellana 298
245,Plaza de San Francisco,Plaza de San Francisco 5
246,Pavones,Calle Hacienda de Pavones 342
247,Parque Santander,Calle Santander esquina con Avenida de Filipinas


In [28]:
df_col_pub_6 = drop_columns_2(df_col_pub_b)
df_col_pub_6 

,Place of interest,Type of place,Place address
0,Colegio Público Adolfo Suárez,Colegio Público,Calle Infanta Catalina Micaela 31
1,Colegio Público Agustina Díez,Colegio Público,Calle Vesubio 5
2,Colegio Público Agustín Rodriguez Sahagún,Colegio Público,Calle La Granja De San Ildefonso 17
3,Colegio Público Alberto Alcocer,Colegio Público,Avenida Canillejas A Vicalvaro 56
4,Colegio Público Alcalde de Móstoles,Colegio Público,Calle Aldeanueva De La Vera 9
...,...,...,...
244,Colegio Público Virgen del Cortijo,Colegio Público,Avenida Manoteras 63
245,Colegio Público Vázquez de Mella,Colegio Público,Calle Bailen 18
246,Colegio Público Winston Churchill,Colegio Público,Avenida Gran Via Del Este 5
247,Colegio Público de Practicas Asunción Rincón,Colegio Público,Avenida Filipinas 5


In [29]:
def concatenate(df_x, df_y):
    nearest_bicimad_cp = pd.concat([df_x, df_y], axis=1)
    return nearest_bicimad_cp

In [30]:
b_nearest_bicimad_cp_1 = concatenate(df_col_pub_6, bicimad_stations_3)
b_nearest_bicimad_cp_1

,Place of interest,Type of place,Place address,BiciMAD station,Station location
0,Colegio Público Adolfo Suárez,Colegio Público,Calle Infanta Catalina Micaela 31,Manuel Caldeiro,Paseo de la Castellana 298
1,Colegio Público Agustina Díez,Colegio Público,Calle Vesubio 5,Sierra Toledana,Camino de Valderribas 94
2,Colegio Público Agustín Rodriguez Sahagún,Colegio Público,Calle La Granja De San Ildefonso 17,Pavones,Calle Hacienda de Pavones 342
3,Colegio Público Alberto Alcocer,Colegio Público,Avenida Canillejas A Vicalvaro 56,Gutierre de Cetina,Calle Gutierre de Cetina 77
4,Colegio Público Alcalde de Móstoles,Colegio Público,Calle Aldeanueva De La Vera 9,Caramuel,Calle Caramuel 51
...,...,...,...,...,...
244,Colegio Público Virgen del Cortijo,Colegio Público,Avenida Manoteras 63,Manuel Caldeiro,Paseo de la Castellana 298
245,Colegio Público Vázquez de Mella,Colegio Público,Calle Bailen 18,Plaza de San Francisco,Plaza de San Francisco 5
246,Colegio Público Winston Churchill,Colegio Público,Avenida Gran Via Del Este 5,Pavones,Calle Hacienda de Pavones 342
247,Colegio Público de Practicas Asunción Rincón,Colegio Público,Avenida Filipinas 5,Parque Santander,Calle Santander esquina con Avenida de Filipinas


In [31]:
list(b_nearest_bicimad_cp_1["BiciMAD station"])

['Manuel Caldeiro',
 'Sierra Toledana',
 'Pavones',
 'Gutierre de Cetina',
 'Caramuel',
 'Pavones',
 'Gutierre de Cetina',
 'Facultad Derecho',
 'Caramuel',
 'Marqués de Zafra',
 'Concordia',
 'Pavones',
 'Pedro Rico',
 'Marqués de Vadillo',
 'Jesús y María',
 'Embajadores 191',
 'Pavones',
 'Facultad Derecho',
 'Marqués de Vadillo',
 'Pío XII',
 'Pavones',
 'Embajadores 191',
 'Embajadores 191',
 'Glorieta de Cádiz',
 'Pedro Rico',
 'Pavones',
 'Caramuel',
 'Francos Rodríguez',
 'Remonta',
 'Gutierre de Cetina',
 'Pedro Rico',
 'Marqués de Vadillo',
 'Pedro Rico',
 'Pavones',
 'Avenida Donostiarra',
 'Gutierre de Cetina',
 'Pavones',
 'Glorieta de los Cuatro Caminos',
 'Gutierre de Cetina',
 'Caramuel',
 'Glorieta de Cádiz',
 'Gutierre de Cetina',
 'Glorieta de Cádiz',
 'Parque Roma',
 'Pavones',
 'Gutierre de Cetina',
 'Glorieta de Cádiz',
 'Parque Santander',
 'Pedro Rico',
 'Marqués de Vadillo',
 'Pavones',
 'C.D. Concepción',
 'Manuel Caldeiro',
 'Caramuel',
 'Glorieta de Cádiz',


In [32]:
def get_csv(df_final):
    return df_final.to_csv('../data/b_nearest_bicimad_cp.csv', index=False, encoding='utf-8')

In [33]:
get_csv(b_nearest_bicimad_cp_1)